In [2]:
from numpy import array
from numpy.linalg import inv, norm
from pandas import DataFrame

In [3]:
def yakobi(x: list, x0):
    lam = x[4]
    return array([[(2 * lam + 1), 0, 2, 0, 2 * x[0] - 2 * x0[0]],
                  [0, (2 * lam + 1), 0, 0, 2 * x[1] - 2 * x0[1]],
                  [2, 0, (2 * lam + 1), 0, 2 * x[2] - 2 * x0[2]],
                  [0, 0, 0, (2 * lam + 1), 2 * x[3] - 2 * x0[3]],
                  [2 * x[0] - 2 * x0[0], 2 * x[1] - 2 * x0[1], 2 * x[2] - 2 * x0[2], 2 * x[3] - 2 * x0[3], 0]])

In [4]:
def f(x: list):
    return 1 / 2 * sum(x[i] ** 2 for i in range(0, 3 + 1)) + 2 * x[0] * x[2] + sum(x[i] for i in range(0, 3 + 1))

In [5]:
def g(x, x0, r):
    return sum((x[i] - x0[i]) ** 2 for i in range(0, 3 + 1)) - r ** 2

In [6]:
def F(x: list, x0, r):
    lam = x[4]
    return f(x) + lam * g(x, x0, r)

In [7]:
def nablaF(x, x0, r):
    lamb = x[4]
    return array([(2 * lamb + 1) * x[0] + 2 * x[2] - 2 * lamb * x0[0] + 1,
                  (2 * lamb + 1) * x[1] - 2 * lamb * x0[1] + 1,
                  (2 * lamb + 1) * x[2] + 2 * x[0] - 2 * lamb * x0[2] + 1,
                  (2 * lamb + 1) * x[3] - 2 * lamb * x0[3] + 1,
                  sum(x[i] ** 2 - 2 * x0[i] * x[i] for i in range(0, 3 + 1)) - r ** 2 + sum(x0[i] ** 2 for i in range(0, 3 + 1))])

In [8]:
def lenNablaF(x, x0, r):
    nabl = nablaF(x, x0, r)
    return norm(nabl)

In [9]:
def x_new(xk, yak, Fxk):
    return xk - inv(yak) @ Fxk

Решения

In [10]:
r_c = 5

lambda_b = 1

point_b = [0, 0, 0, 0]

eps_r = 1

eps = 0.001

df = DataFrame(columns=["point", "f(x)", "count_iter"])

for a_c in range(point_b[0] - r_c - eps_r, point_b[0] + r_c + eps_r + 1, 2):
    for b_c in range(point_b[1] - r_c - eps_r, point_b[1] + r_c + eps_r + 1, 2):
        for c_c in range(point_b[2] - r_c - eps_r, point_b[2] + eps_r + 1, 2):
            for d_c in range(point_b[3] - r_c - eps_r, point_b[3] + r_c + eps_r + 1, 2):
                count_iter = 0
                point = [a_c, b_c, c_c, d_c, lambda_b]
                try:
                    while lenNablaF(point, point_b, r_c) > eps:
                        yak = yakobi(point, point_b)
                        Fcur = nablaF(point, point_b, r_c)
                        point = x_new(point, yak, Fcur)
                        count_iter += 1
                    df.loc[df.shape[0]] = [point, f(point), count_iter]
                    # print(point, f(point))
                except:
                    pass
df.sort_values(['f(x)'])

,point,f(x),count_iter
1222,"[3.241129226604991, -0.5000016685032199, -3.74...",-13.250489,4
1330,"[3.2411292266049907, -0.5000023720356377, -3.7...",-13.250489,4
652,"[3.241130342323241, -0.49998112891892654, -3.7...",-13.250449,5
274,"[-3.7411249284542603, -0.49998112891892654, 3....",-13.250449,5
247,"[-3.7411249284542603, -0.49997454510007566, 3....",-13.250449,5
...,...,...,...
891,"[3.5083621377522443, 0.4376079490637372, 3.508...",45.009338,11
688,"[3.5083647230408253, 0.43763213043308086, 3.50...",45.009384,10
472,"[3.5083647253970756, 0.4376317311705609, 3.508...",45.009384,10
499,"[3.5083647253970756, 0.4376321304330809, 3.508...",45.009384,10


In [11]:
df.shape[0]

1371

In [12]:
df['f(x)'].median()

np.float64(-13.250000078769121)

In [13]:
point = [point_b[0], point_b[1]+ r_c, point_b[2], point_b[3], lambda_b]
while lenNablaF(point, point_b, r_c) > eps:
    yak = yakobi(point, point_b)
    Fcur = nablaF(point, point_b, r_c)
    point = x_new(point, yak, Fcur)
print(point, f(point))

[-0.58368759  3.48694356 -0.58368759  3.4871468  -0.64338227] 18.98827290258032
